# Import libraries

In [1]:
import sys
sys.path.append("../")
from ortho_lib3_Copy3 import *
import pandas as pd
import numpy as np
import copy

# Define functions and class

In [2]:
# calculates the surface between sensor 1 and sensor 2 (right and left) and divides by the total amount of frames
# pass: dataframe, axis (x, y, or z), sensor 1 and sensor 2 (right and left side)
# returns: difference of surface divided by the amount of frames (.max() gives the maximum assymmerty calculated)
def get_surf_diff(df, ax, s_1, s_2):
    return df[ax + '_' + s_1] - df[ax + '_' + s_2]

# filter out patients that have not completed all exercises
def filter_incomplete_patients(exercises, exercises_types = ['AF', 'AB', 'EL', 'RF']):
    new_exercises = copy.deepcopy(exercises)
    for patient in exercises.patients:
        keys = exercises[patient].keys()
        for ex in exercises_types:
            if ex not in keys:
                new_exercises = new_exercises.drop_patient(patient)
                print('dropped ' + str(patient) + ' because there is (a) missing exercise(s): ', list(keys))
    return new_exercises

# create class with features
class ExtendedDF(DFFrame):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self['surf_diff_elbows_x_l'] = get_surf_diff(self, 'x', '5').abs()
        self['surf_diff_elbows_x_r'] = get_surf_diff(self, 'x', '8').abs()
        self['surf_diff_elbows_y_l'] = get_surf_diff(self, 'y', '5').abs()
        self['surf_diff_elbows_y_r'] = get_surf_diff(self, 'y', '8').abs()
        self['surf_diff_elbows_z_l'] = get_surf_diff(self, 'z', '5').abs()
        self['surf_diff_elbows_z_r'] = get_surf_diff(self, 'z', '8').abs()
                

# Create usable dataframe

In [3]:
dfs = create_dfframes([2,3,4], 
                extype= ['AB', 'AF', 'EL', 'RF'], 
                data_dir = '../transformed_data', 
                print_errors = False, 
                show_progress = True)

# Add features to patient

In [4]:
Patient.add_features(['diff_x_elbow'], ['AB'], ['max'])
Patient.add_features(['diff_y_elbow'], ['AB'], ['max'])
Patient.add_features(['diff_z_elbow'], ['AB'], ['max'])

exercises = dffs_to_exercises(dfs)
exercises = filter_incomplete_patients(exercises)

dropped 441 because there is (a) missing exercise(s):  ['AB', 'EL', 'RF']
dropped 423 because there is (a) missing exercise(s):  ['AB', 'AF', 'RF']
dropped 421 because there is (a) missing exercise(s):  ['AB', 'AF', 'RF']


# Run experiment

In [5]:
exp = Experiment(exercises, y_condition = lambda cat: cat != 'Category_2')

# Retreive results of experiment for specific factor

In [6]:
factor = 1.1
results = exp.fit_inliers_ensemble(['diff_x_elbow_max_AB'], factor)
results = exp.fit_inliers_ensemble(['diff_y_elbow_max_AB'], factor, results=results)
results = exp.fit_inliers_ensemble(['diff_z_elbow_max_AB'], factor, results=results)

In [7]:
results.report()

,features,recall,precision,accuracy
0,diff_y_elbow_max_AB,0.0,0.0,0.360825
1,diff_x_elbow_max_AB,0.0,0.0,0.360825
2,diff_z_elbow_max_AB,0.0,0.0,0.371134
